# LIBRARIES IMPORT

In [3]:
# imports

import fig_lib 
import CNN_lib
import CNN_utilities
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import pickle

from tensorflow import keras
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split


In [16]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [10]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18360024797699070600
xla_global_id: -1
]


In [14]:
init_time = time.time()

In [15]:
# p
pkl_d_p = 'C:\PROJECTS\emboendo\CNN/input_d.pkl'
pkl_train_p='C:\PROJECTS\emboendo\CNN/train_d.pkl'        

In [ ]:
# ------- PROCESS DICOM DATA --------------

data_load_dicom,generate_new_peakle=True,True

max_frames_allowed=40

# ------- GENERATE SYNTHETIC DATA --------------

generate_new_data=False 


# --------- LOAD TRAN DATA ------------

load_train_data=False

# --------- DATA TYPE ------------
video_d=True                                    # Video = True , Image = False 

# DATA LOAD

In [ ]:

if data_load_dicom:

    patients_recs_d_df=CNN_utilities.main_d_df()
    patients_labels_d_df=CNN_utilities.labels_df()
    patients_d_df=pd.merge( patients_labels_d_df,patients_recs_d_df, on='PatientID')
    

    tot_recs=sum(len(records) for records in patients_d_df["Records"])
    filtered_df_0,filtered_df_1 = patients_d_df[patients_d_df['label'] == 0],patients_d_df[patients_d_df['label'] == 1]
    label_0,label_1 = sum(len(records) for records in filtered_df_0['Records']),sum(len(records) for records in filtered_df_1['Records'])

    print(f'- PATIENTS: {patients_d_df.shape[0]} ,PATIENT LABEL 0: {sum(patients_d_df["label"]==0)}, PATIENT LABEL 1: {sum(patients_d_df["label"]==1)}')
    print( f'- RECORDS: {tot_recs} , RECORDS LABEL 0: {label_0}, RECORDS LABEL 1:{label_1}')
    print(f'- RATIO LABEL 0: {np.round(label_0/sum(patients_d_df["label"]==0),decimals=2)}, RATIO LABEL 1: {np.round(label_1/sum(patients_d_df["label"]==1),decimals=2)}')

    display(patients_d_df)


In [ ]:

if data_load_dicom:
    
    if generate_new_peakle:
        
        input_d={}    

        for label,ii in zip(patients_d_df['label'],patients_d_df['Records']):
            
            for rec in ii:

                input_d[rec]={'label':label}
                ds_file=CNN_utilities.load_rec(rec)
                vd_ls=[]

                for i in range(ds_file.pixel_array.shape[0]):
                    
                    if i== max_frames_allowed: break
                    else:
                        image_8bit,  width, height = CNN_utilities.im_data(ds_file.pixel_array[i])
                        vd_ls.append(image_8bit)
                    

                input_d[rec]['image']=np.array(vd_ls)
                input_d[rec]['dimHW']= [height,width]   

        with open(pkl_d_p, 'wb') as pikle_file:
            pickle.dump(input_d, pikle_file)
        

    else: 

        with open(pkl_d_p, 'rb') as pikle_file:
            input_d = pickle.load(pikle_file)
       


In [ ]:
if data_load_dicom:    
    
    print('FIRST LAYER KEYS:', list(input_d.keys()))
    print('SECOND LAYER KEYS:',list(input_d[list(input_d.keys())[0]].keys()))
    print(f' KEY TYPE "{list(input_d[list(input_d.keys())[0]].keys())[0]}":',type(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[0]].keys())[0]]),'Example:',input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[0]].keys())[0]])
    print(f' KEY TYPE "{list(input_d[list(input_d.keys())[0]].keys())[1]}":',type(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[1]].keys())[1]]),'Example shape:',np.shape(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[1]].keys())[1]]))
    


In [ ]:
if data_load_dicom:
    print('RECORD - LABEL - SHAPE')

    for R in input_d.keys():
        print(R,input_d[R]['label'],np.shape(input_d[R]['image']),type(input_d[R]['image']))
    

# DATA GENERATOR

In [ ]:
# -------- DATA PARAMS --------               

if generate_new_data:    

    total_n_recs=400
    #n_recs_train,n_recs_eval=300,80
    HEIGHT, WIDTH = 100, 100
    circles_filled=False
    color=255

    # -------- Video ---------
 

    n_frames=[10,12]    # [min,max]

    t_btw_frm=0.2

In [ ]:
#input_d, eval_d

if generate_new_data:

    if video_d:
        
        input_d=fig_lib.vid_data_gen(HEIGHT, WIDTH,total_n_recs,n_frames,'I',cir_f=circles_filled,color=color)
        
    else:

        input_d=fig_lib.im_data_gen(HEIGHT, WIDTH,total_n_recs,'I',cir_f=circles_filled,color=color)

    print('FIRST LAYER KEYS:', list(input_d.keys()))
    print('SECOND LAYER KEYS:',list(input_d[list(input_d.keys())[0]].keys()))
    print(f' KEY TYPE "{list(input_d[list(input_d.keys())[0]].keys())[0]}":',type(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[0]].keys())[0]]),'Example:',input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[0]].keys())[0]])
    print(f' KEY TYPE "{list(input_d[list(input_d.keys())[0]].keys())[1]}":',type(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[1]].keys())[1]]),'Example shape:',np.shape(input_d[list(input_d.keys())[0]][list(input_d[list(input_d.keys())[1]].keys())[1]]))


In [ ]:
print('RECORD - LABEL - SHAPE')

if generate_new_data:
    
    for R in input_d.keys():
        print(R,input_d[R]['label'],np.shape(input_d[R]['image']),type(input_d[R]['image']))

# CHECK DATA

In [ ]:
t_btw_frm=0.05
d_plt_check,rec_check=False,'I0'
print(input_d[rec_check]['label'])


In [ ]:
# check video

if d_plt_check:

    if video_d:fig_lib.vid_show(input_d[rec_check]['image'],t_btw_frm=t_btw_frm)
    else:fig_lib.im_show(input_d[rec_check]['image'])



# CNN

### GENERATE TRAIN DATA

In [ ]:
# X_train,Y_train

if load_train_data: 
      
    with open(pkl_train_p, 'rb') as pikle_file:

        save_trains= pickle.load(pikle_file)

    X_train,Y_train =  save_trains[0],save_trains[1]

else:

    if video_d:X_train,Y_train,max_frm_n,HEIGHT, WIDTH =CNN_lib.vid_d_bin_gen(input_d, zero=0, one= 1)
    else:X_train,Y_train =CNN_lib.im_d_bin_gen(input_d,zero='circle', one= 'line')

    save_trains=[X_train,Y_train]

    with open(pkl_train_p, 'wb') as pikle_file:
            
            pickle.dump(save_trains, pikle_file)



In [ ]:
# X_train,Y_train,X_eval,Y_eval

test_size=0.2
X_train, X_eval, Y_train, Y_eval = train_test_split(X_train, Y_train, test_size=test_size) #,random_state=42)

print('Train data:',len(Y_train),'Evaluation data:',len(Y_eval))

In [ ]:
# check data

zeros_count = np.sum(Y_train == 0)
ones_count = np.sum(Y_train == 1)
zeros_count_val = np.sum(Y_eval == 0)
ones_count_val = np.sum(Y_eval == 1)

labels = ['Circles Train', 'Lines Train','Circles Validation', 'Lines Validation']
print(' Train data shape:', np.shape(X_train),' Validation data shape:', np.shape(X_eval))

plt.bar(labels, [zeros_count, ones_count,zeros_count_val,ones_count_val], color=['green', 'blue','green', 'blue'])

for i, count in enumerate([zeros_count, ones_count,zeros_count_val,ones_count_val]):

    plt.text(i, count + 0.1, str(count), ha='center', va='bottom')

plt.title('Data Labels')
plt.show()

In [ ]:
end_time = time.time()

process_time = end_time - init_time
print(f"Tiempo transcurrido: {process_time} segundos")

### PARAMETERS

In [ ]:
print('LIBRARY MODELS:')
print(' ')

for model_name in CNN_lib.MODELS.values(): print(model_name)

In [ ]:
# -------- CNN PARAMS --------

# -------- MODEL --------

model_from_CNN_lib,mdl= False,'video_conv2D'

opt='adam'
lss='binary_crossentropy'

# -------- TRAINNING --------

epch=2


### MODEL

In [ ]:
# MODEL 

if model_from_CNN_lib:
  
  if video_d: model = CNN_lib.lib_models(mdl,im_input_shp=(max_frm_n, HEIGHT, WIDTH, 1))
  else: model = CNN_lib.lib_models(mdl,im_input_shp=(HEIGHT, WIDTH,1))

else:

  model = models.Sequential([
          layers.Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu',input_shape=(max_frm_n, HEIGHT, WIDTH, 1)),
          layers.MaxPooling3D(pool_size=(2, 2, 2)),
          layers.Conv3D(filters=32, kernel_size=(1, 3, 3),  activation='relu'),
          layers.MaxPooling3D(pool_size=(2, 2, 2)),
          layers.Conv3D(filters=32, kernel_size=(1, 3, 3),  activation='relu'),
          layers.MaxPooling3D(pool_size=(2, 2, 2)),
          layers.Conv3D(filters=64, kernel_size=(1, 3, 3),  activation='relu'),
          layers.Flatten(),
          layers.Dense(64, activation='relu'),
          layers.Dense(1,activation='sigmoid')
          ])


model.summary()

In [ ]:
model.compile(optimizer=opt, loss=lss, metrics=['accuracy'])

### TRAINNING

In [ ]:

history =model.fit(X_train, Y_train, epochs=epch, validation_data=(X_eval,Y_eval))


### EVALUATE MODEL

In [ ]:
fg_size=(14,8)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epch)

plt.figure(figsize=fg_size)
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')  
plt.ylabel('Accuracy')  

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')  
plt.ylabel('Loss')  
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_eval,Y_eval,verbose=2)

print(f'Loss: {test_loss}',f'Accuracy: {test_accuracy}')

In [ ]:
performance_d={'NAME':mdl,
               'ACCURACY':test_accuracy,
               'LOSS':test_loss,
               'VIDEO':video_d,
               'TRAIN SAMPLES':n_recs_train,
               'VALIDATION SAMPLES':n_recs_eval,
               'TRAINNING TIME':t_train,
               'EPOCHS':epch,
               'FRAMES RANGE':n_frames,
               'HEIGHT':HEIGHT,
               'WIDTH':WIDTH}

performance_d

### FORECAST

GENERATE RANDOM IMAGE

In [ ]:
# generate random im

n_rdm=np.random.rand()

if video_d:
    
    vd_ls=[]
    N_frames=np.random.randint(n_frames[0],n_frames[1]+1)

    if n_rdm>=0.5:

        p1=[np.random.randint(0, WIDTH-1),np.random.randint(0, HEIGHT-1)]
        p2=[np.random.randint(0, WIDTH-1),np.random.randint(0, HEIGHT-1)]

        for _ in range(N_frames):

            vd_ls.append(fig_lib.line_im(p1,p2,height=HEIGHT, width=WIDTH,thickness=(1,3),color=color))
            p1[0]+= np.random.randint(-int(WIDTH*0.02), int(WIDTH*0.02))
            p2[0]+=np.random.randint(-int(WIDTH*0.06), int(WIDTH*0.06))
            p1[1]+= np.random.randint(-int(HEIGHT*0.02), int(HEIGHT*0.02))
            p2[1]+=np.random.randint(-int(HEIGHT*0.06), int(HEIGHT*0.06))

        label='Line'
        

    else:

        center= [np.random.randint(int(0+WIDTH*0.3), int(WIDTH - WIDTH*0.3)),np.random.randint(int(0+HEIGHT*0.3), int(HEIGHT-HEIGHT*0.3))]
        radio = np.random.randint(12, 17)

        for _ in range(N_frames):
        
            vd_ls.append(fig_lib.circle_im(center,radio,height=HEIGHT, width=WIDTH,thickness=(1,3),color=color,filled=circles_filled))
            center[0]+= np.random.randint(-int(WIDTH*0.03), int(WIDTH*0.03))
            center[1]+= np.random.randint(-int(HEIGHT*0.03), int(HEIGHT*0.03))
            radio += np.random.randint(-int(HEIGHT*0.01), int(HEIGHT*0.02))

        label ='circle'

    video=np.array(vd_ls)
    image = video[:, :, :, np.newaxis]
    frames_actual = image.shape[0]
    if frames_actual < max_frm_n:
        
        padding = np.zeros((max_frm_n - frames_actual, HEIGHT, WIDTH, 1))
        image = np.concatenate([image, padding], axis=0)

else:

    if n_rdm>=0.5:

        p1=[np.random.randint(0, WIDTH-1),np.random.randint(0, HEIGHT-1)]
        p2=[np.random.randint(0, WIDTH-1),np.random.randint(0, HEIGHT-1)]
        image=fig_lib.line_im(p1,p2,height=HEIGHT, width=WIDTH,thickness=(1,3),color=color)
        label='Line'

    else:
        
        center= [np.random.randint(int(0+WIDTH*0.3), int(WIDTH - WIDTH*0.3)),np.random.randint(int(0+HEIGHT*0.3), int(HEIGHT-HEIGHT*0.3))]
        radio = np.random.randint(12, 17)
        image=fig_lib.circle_im(center,radio,height=HEIGHT, width=WIDTH,thickness=(1,3),color=color,filled=circles_filled)
        label= 'circle'

In [ ]:
# Forecast

pred = model.predict(np.expand_dims(image, axis=0),verbose=2)
CNN_lib.bin_forecast(pred,label=label,up_frc='line',down_frc='circle')


In [ ]:
# show

if video_d:fig_lib.vid_show(video)
else:fig_lib.im_show(image)